In [1]:
from ML_utils import ML_utils
from gensim.models import word2vec
from gensim.models import KeyedVectors
import shutil
import os
import numpy as np
import pandas as pd
import re
import logging
import sys

# train wordvec

In [ ]:
file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki_tw.txt"
ML_utils.Opencc_file(file_path,save_path,replace_old=True)

In [ ]:
userdict_path = './jieba_dict/ptt_userdict.txt'
jieba_dict = 'jieba_dict/jieba_merge.txt'
dir_path = r"./word2vec_model"
ML_utils.CallF_DirFile_save(dir_path, ML_utils.Jieba_file_segmentation, 
                            replace_old=False,regular=True,file_filter_=None,
                            dict_path=jieba_dict,load_userdict_path=userdict_path,HMM=False)

In [ ]:
dir_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
ML_utils.Word2vec_train(file_path=None,dir_path=dir_path,save_path=save_path,model_min_count=1)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='台灣',topn=20)

In [19]:
a = model.wv.get_vector('')
print(max(a),min(a))
a

3.14634 -3.2558982


array([ 2.19233370e+00, -1.46388721e+00, -1.99675608e+00,  6.79616570e-01,
        3.55416924e-01,  1.59947169e+00,  2.12003097e-01,  1.02245383e-01,
       -6.30102754e-02,  1.45580783e-01, -1.20840502e+00,  1.05694366e+00,
       -4.72883910e-01, -7.30408907e-01, -6.62375212e-01,  3.69489416e-02,
        6.03886724e-01, -1.04588306e+00, -9.98719633e-01,  4.87945765e-01,
       -1.19801438e+00,  5.29940845e-03, -5.62464714e-01,  9.90700066e-01,
       -1.11428118e+00,  2.33197063e-02, -4.84015942e-01, -4.22614723e-01,
        1.55693221e+00, -8.42484772e-01,  1.12706840e+00, -4.90916669e-01,
       -1.82026911e+00,  1.78608343e-01,  1.95130181e+00, -5.35292745e-01,
       -2.13389695e-01, -4.45280492e-01, -6.62543356e-01, -2.24680066e-01,
       -8.58971775e-01,  3.13481003e-01, -4.12401855e-01,  8.42198312e-01,
        8.90280306e-01, -2.93085545e-01,  5.73269837e-02,  5.64253807e-01,
        1.94658458e-01,  7.06532478e-01, -9.69610810e-01, -2.71015286e-01,
       -1.37021288e-01,  

# word2vec add token

In [ ]:
#add token
model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
train_file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\jieba_dict\token\token.txt"
ML_utils.add_word2vec_word(model_path,save_path,train_file_path=train_file_path)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='__EOS.Token__',topn=20)

# data add token

In [ ]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
ML_utils.data_add_token_SOS_EOS(data_path,save_path)

# data jieba segmentation

In [ ]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
load_userdict_path = './jieba_dict/ptt_userdict.txt'
dict_path = 'jieba_dict/jieba_merge.txt'
ML_utils.Jieba_file_segmentation(data_path,save_path,
                                 dict_path=dict_path,load_userdict_path=load_userdict_path,HMM=False)

# add loss_word to wordvec

In [ ]:
def Null_to_space(list_in):
    index = []
    for num in range(len(list_in)):        
        remove_num = 0
        if list_in[num] == "" or list_in[num] == "\u3000":
            index.append(num)
    for num in range(len(index)):
        remove_index = index[num]-remove_num
        list_in.pop(remove_index)
        remove_num = remove_num+1
    return list_in

def get_pair_vec(model,pairs,num):
    pair = pairs[num]
    X = pair[0]
    Y = pair[1]
    loss_word = []
    try:
        for word in X:
            model.wv.get_vector(word)
        for word in Y:
            model.wv.get_vector(word)
    except:
        loss_word.append(word)
    return loss_word

data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
pairs = []
with open(data_path,"r",encoding="utf-8") as f:
    i = 0
    for line in f:
        pair = line.split("\t")
        X = pair[0].strip(" ").split(" ")
        X = Null_to_space(X)
        Y = pair[1].strip("\n").strip(" ").split(" ")
        Y = Null_to_space(Y)
        pairs.append([X,Y])

model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
model = word2vec.Word2Vec.load(model_path)
loss_word_set = set()
for num in range(len(pairs)):
    loss_word = get_pair_vec(model,pairs,num)
    loss_word_set.update(loss_word)

sentences = []   
for word in list(loss_word_set):
    sentences.append([word])
print(sentences)    
model.vocabulary.min_count = 1
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model.build_vocab(sentences, update=True)#训练该行
model.train(sentences,total_examples= model.corpus_count,epochs= model.epochs)
model.save(save_path)
model.wv.most_similar(positive=sentences[0],topn=5)

# wordvec save keyvector

In [ ]:
model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\keyvector\model_keyvector"
ML_utils.wordvec2keyvector(model_path,save_path)

In [ ]:
#test
model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\keyvector\model_keyvector"
word_vectors = KeyedVectors.load(model_path, mmap='r')
test = word_vectors.get_vector("hello")
word_vectors.similar_by_vector(test)

# test clip data

In [ ]:
def del_space(list_in):
    index = []
    for num in range(len(list_in)):        
        remove_num = 0
        if list_in[num] == "" or list_in[num] == "\u3000":
            index.append(num)
    for num in range(len(index)):
        remove_index = index[num]-remove_num
        list_in.pop(remove_index)
        remove_num = remove_num+1
    return list_in

def file_to_word_list(data_path):
    pairs = []
    with open(data_path,"r",encoding="utf-8") as f:
        i = 0
        for line in f:
            pair = line.split("\t")
            X = pair[0].strip(" ").split(" ")
            X = del_space(X)
            Y = pair[1].strip("\n").strip(" ").split(" ")
            Y = del_space(Y)
            pairs.append([X,Y])
            if len(X) > 22:
                print(X)
            if len(Y) > 22:
                print(Y)
    return pairs
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
pairs = file_to_word_list(data_path)